# Imports

In [132]:
from pathlib2 import Path
from riotwatcher import LolWatcher, ApiError

import arrow
import json
import pandas as pd

# Functions

In [112]:
def api_key(credentials):
    """Checking if the development API key from riot's development portal has been set and/or needs to be updated
    
    Input is a JSON file 

    """
    with open(credentials) as json_data:
        credentials = json.load(json_data)
    return credentials["dev_api_key"]
    # TODO: add that it checks for if the key is still valid or not error 404 or something and if so enter it and it updates the dev_api_key.json file
    # global api_key
    # try:
    #     api_key
    # except NameError:
    #     api_key = str(input("Input development API key."))
    # else:
    #     print("API key already defined.")
    

# Main

In [113]:
# Folder locations

# Project folders
project_folder = Path.cwd().parent

# Raw data storage
data_folder = project_folder / "data"

# Out folder
out_folder = project_folder / "out"

In [114]:
# Set API key
api_key_loc = data_folder / "dev_api_key.json"
lol_watcher = LolWatcher(api_key(api_key_loc)) 

In [121]:
# Set user request parameters

# Set region
region = "euw1"

# Set summonder name 
summoner_name = "alnas" # not case sensitive

### User info

In [118]:
# retrieve info using region and summoner name

user = lol_watcher.summoner.by_name(region, summoner_name)
print(user)

{'id': 'vvnJnOwimLDEuJz55em8ihuAIjfQcfQk0qNTs1xK1SujIEA', 'accountId': 'U8waP8B_8dhhTEMZOZtmyV4vqBfEYHiCNnUMXmPD4EaMeQ', 'puuid': '-0mFh1iO9XEjqTEh8xofNi7wsCx_I7sLHsxhSp7sv8q4sIDuQa_ei3MtfGkud7KyX9PO_ujw76m5yg', 'name': 'Alnas', 'profileIconId': 3382, 'revisionDate': 1608752230000, 'summonerLevel': 101}


In [119]:
# Check rank stats using user id
ranked_stats = lol_watcher.league.by_summoner(region, user["id"])
print(ranked_stats)

[{'leagueId': 'e74bb70a-b493-4f52-a031-c20909161b03', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'GOLD', 'rank': 'II', 'summonerId': 'vvnJnOwimLDEuJz55em8ihuAIjfQcfQk0qNTs1xK1SujIEA', 'summonerName': 'Alnas', 'leaguePoints': 25, 'wins': 46, 'losses': 41, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}]


### Match info

In [ ]:
# set match request parameters

# Set queue
queue = 420

# Set season

In [134]:
# Retrieve last 100 matches (max is 100 matches https://developer.riotgames.com/apis#match-v4) but time range is 1 week 
matches = lol_watcher.match.matchlist_by_account(region=region,
                                                 encrypted_account_id=user["accountId"],
                                                 queue=queue,
                                                 begin_index=0,
                                                 end_index=100)
# print(matches.keys())

# Get match details
for i, match in enumerate(matches["matches"][0:1]): # currently last 5 matches
        print(i, match)
        match_detail = lol_watcher.match.by_id(region, match['gameId'])
        
participants = []
for row in match_detail['participants']:
    participants_row = {}
    participants_row['champion'] = row['championId']
    participants_row['spell1'] = row['spell1Id']
    participants_row['spell2'] = row['spell2Id']
    participants_row['win'] = row['stats']['win']
    participants_row['kills'] = row['stats']['kills']
    participants_row['deaths'] = row['stats']['deaths']
    participants_row['assists'] = row['stats']['assists']
    participants_row['KDA'] = (row['stats']['kills']+row['stats']['assists'])/row['stats']['deaths']
    participants_row['totalDamageDealt'] = row['stats']['totalDamageDealt']
    participants_row['goldEarned'] = row['stats']['goldEarned']
    participants_row['champLevel'] = row['stats']['champLevel']
    participants_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
    participants_row['item0'] = row['stats']['item0']
    participants_row['item1'] = row['stats']['item1']
    participants.append(participants_row)
df = pd.DataFrame(participants)
df

0 {'platformId': 'EUW1', 'gameId': 4994448524, 'champion': 202, 'queue': 420, 'season': 13, 'timestamp': 1608750682906, 'role': 'SOLO', 'lane': 'BOTTOM'}


,champion,spell1,spell2,win,kills,deaths,assists,KDA,totalDamageDealt,goldEarned,champLevel,totalMinionsKilled,item0,item1
0,245,14,4,False,3,12,2,0.416667,86328,6624,12,117,2033,3152
1,202,4,7,False,2,7,6,1.142857,52629,7906,11,113,1055,3009
2,75,4,12,False,6,4,6,3.000000,82125,10014,14,130,1054,3110
3,28,4,11,False,9,9,3,1.333333,117825,10215,13,20,1082,3100
4,99,4,14,False,2,5,4,1.200000,42003,6743,11,63,3860,4005
5,101,4,14,True,10,5,3,2.600000,48057,9959,12,40,3853,3020
6,777,4,14,True,12,5,8,4.000000,165417,12944,16,175,1055,6673
7,33,4,11,True,5,4,16,5.250000,97872,9997,14,14,3068,3111
8,67,4,12,True,5,6,10,2.500000,87816,10465,14,134,1083,3009
9,145,4,7,True,5,2,8,6.500000,76364,10332,13,159,1055,6671


### Patch an season info

In [110]:
# example of how to Read time
with open(data_folder / "patches.json") as f:
    data = json.load(f)

patch = data['patches'][-1]  # most recent
utc_timestamp = patch['start']
north_america_timestamp = utc_timestamp + data['shifts']['NA1']
print(north_america_timestamp)
dt = arrow.get(north_america_timestamp)
print(dt.to('US/Pacific'))

1607515200
2020-12-09T04:00:00-08:00


# Calculating KDA


In [ ]:
Defining variables

KDA_Match: KDA calculated per match per player
KDA_Season: Mean KDA for that player for this seasons

K_Match: Total kills in that match
D_Match: Total deaths in that match
A_Match: Total assists in that match

K_Season: Total kills in that season
D_Season: Total deaths in that season
A_Season: Total assists in that season
? Should we use total or average?

Calculations

KDA base calculation= Kills + Assists / Deaths
KDA_Match = K_Match + A_Match / D_Match
KDA_Season = K_Season + A_Season / D_Season


Setting things up: 
Season: sets the season we are pulling data from
Region: sets the region we are pulling data from



In [7]:
Confounding variables

Rank


0 10.0
1 9.900990099009901
2 9.803921568627452
3 9.70873786407767
4 9.615384615384615
5 9.523809523809524
6 9.433962264150944
7 9.345794392523365
8 9.25925925925926
9 9.174311926605505
10 9.090909090909092
11 9.00900900900901
12 8.928571428571429
13 8.849557522123893
14 8.771929824561404
15 8.695652173913043
16 8.620689655172415
17 8.547008547008547
18 8.474576271186441
19 8.403361344537815
20 8.333333333333334
21 8.264462809917354
22 8.19672131147541
23 8.130081300813009
24 8.064516129032258
25 8.0
26 7.936507936507937
27 7.874015748031496
28 7.8125
29 7.751937984496124
30 7.6923076923076925
31 7.633587786259542
32 7.575757575757576
33 7.518796992481203
34 7.462686567164179
35 7.407407407407407
36 7.352941176470588
37 7.299270072992701
38 7.246376811594203
39 7.194244604316546
40 7.142857142857143
41 7.092198581560283
42 7.042253521126761
43 6.993006993006993
44 6.944444444444445
45 6.896551724137931
46 6.8493150684931505
47 6.802721088435374
48 6.756756756756757
49 6.7114093959731544

In [ ]:
# example end script for wayyy later

try:
    response = lol_watcher.summoner.by_name(region, summoner_name)
    print(response)
except ApiError as error:
    if error.response.status_code == 429:
        print('Request limit reached retry in {} seconds.'.format(err.headers['Retry-After']))
    elif error.response.status_code == 404: 
        print("Found no match")